## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [1]:
from arcgis.gis import GIS
gis = GIS("home")
from arcgis.features import FeatureLayerCollection
import requests
import pandas as pd

# Acces to FeatureLayerCollection

In [2]:
item_id = 'd273b3bd14cd4406b43da840619f824f'
cities_id = '3a2fd0ff63d94aaba14d50b37be13d37'
pm10_id = '0875f40418db4224998c82516fd7bcd0'
pm25_id = '640d787469544485b391a7fd3d04e588'
so2_id = 'e3edfb4e955a4040ba494fd0352cb913'
o3_id = 'f5592d9aed144de1830345cfa5aabf94'
no2_id = 'cfc2e00ede524e3485635f9c5589a1ad'


data_item = gis.content.get(item_id)
cities_item = gis.content.get(cities_id)
pm10_item = gis.content.get(pm10_id)
pm25_item = gis.content.get(pm25_id)
so2_item = gis.content.get(so2_id)
o3_item = gis.content.get(o3_id)
no2_item = gis.content.get(no2_id)


flayercol = FeatureLayerCollection.fromitem(data_item)
flc_cities = FeatureLayerCollection.fromitem(cities_item)
flc_pm10 = FeatureLayerCollection.fromitem(pm10_item)
flc_pm25 = FeatureLayerCollection.fromitem(pm25_item)
flc_so2 = FeatureLayerCollection.fromitem(so2_item)
flc_o3 = FeatureLayerCollection.fromitem(o3_item)
flc_no2 = FeatureLayerCollection.fromitem(no2_item)

# Tables for data 

In [3]:
#preparing tables for data
sensor_df = pd.DataFrame()
aq_df = pd.DataFrame()
measurement_df = pd.DataFrame()
chart_data_df = pd.DataFrame()
pm10_df = pd.DataFrame()
pm10_pom_df = pd.DataFrame()
pm25_df = pd.DataFrame()
pm25_pom_df = pd.DataFrame()
o3_df = pd.DataFrame()
o3_pom_df = pd.DataFrame()
no2_df = pd.DataFrame()
no2_pom_df = pd.DataFrame()
so2_df = pd.DataFrame()
so2_pom_df = pd.DataFrame()

# Station data + Air quality Index

In [4]:
#station data
station="https://api.gios.gov.pl/pjp-api/rest/station/findAll"
station_response = requests.get(station)
station_json = station_response.json()
station_df = pd.json_normalize(station_json)

#ordering naming in station_df
station_df.rename(columns={'id':'stationId'}, inplace=True)

for idx in station_df.index:
    #sensor data
    sensor = "https://api.gios.gov.pl/pjp-api/rest/station/sensors/" + str(station_df['stationId'][idx])
    sensor_response = requests.get(sensor)
    sensor_json = sensor_response.json()
    sdf = pd.json_normalize(sensor_json)
    sensor_df = pd.concat([sensor_df, sdf])

    #air quality index
    aq = "https://api.gios.gov.pl/pjp-api/rest/aqindex/getIndex/" + str(station_df['stationId'][idx])
    aq_response = requests.get(aq)
    aq_json = aq_response.json()
    aqdf = pd.json_normalize(aq_json)
    aq_df = pd.concat([aq_df, aqdf])


#ordering naming and indexing in sensor_df
sensor_df.rename(columns={'id':'sensorId'}, inplace=True)
sensor_df.reset_index(drop=True, inplace=True)

#inner join station_df and sensor_df on stationId
ss_df = pd.merge(station_df, sensor_df, on='stationId')

#ordering naming and indexing in measurement_df
aq_df.rename(columns={'id':'stationId'}, inplace=True)
aq_df.reset_index(drop=True, inplace=True)
aq_df.fillna('Brak danych', inplace=True)

#inner join ss_df and aq_df on stationId
ssaq_df = pd.merge(ss_df, aq_df, on='stationId')


# Measuremets 

In [5]:
#measurement data from last 1h
for idx in sensor_df.index:
    measurement = "https://api.gios.gov.pl/pjp-api/rest/data/getData/" + str(sensor_df['sensorId'][idx])
    measurement_response = requests.get(measurement)
    measurement_json = measurement_response.json()
    mdf = pd.json_normalize(measurement_json)
    #measurement data from the last hour
    mvdf = pd.DataFrame(mdf['values'][0]).iloc[:1]
    mvdf['sensorId'] = [sensor_df['sensorId'][idx]]
    measurement_df = pd.concat([measurement_df, pd.concat([mdf['key'], mvdf], axis=1)])
    #measurment data from last hours/days
    chart_data = pd.DataFrame.from_dict(mdf.iloc[:,1][0])
    if mdf['key'][0] == 'O3':
        o3_pom_df = chart_data
        o3_pom_df.rename(columns={'value':sensor_df['sensorId'][idx].astype(str)}, inplace=True)
        o3_df = pd.concat([o3_df, o3_pom_df], axis=1)
    elif mdf['key'][0] == 'NO2':
        no2_pom_df = chart_data
        no2_pom_df.rename(columns={'value':sensor_df['sensorId'][idx].astype(str)}, inplace=True)
        no2_df = pd.concat([no2_df, no2_pom_df], axis=1)
    elif mdf['key'][0] == 'SO2':
        so2_pom_df = chart_data
        so2_pom_df.rename(columns={'value':sensor_df['sensorId'][idx].astype(str)}, inplace=True)
        so2_df = pd.concat([so2_df, so2_pom_df], axis=1)
    elif mdf['key'][0] == 'PM2.5':
        pm25_pom_df = chart_data
        pm25_pom_df.rename(columns={'value':sensor_df['sensorId'][idx].astype(str)}, inplace=True)
        pm25_df = pd.concat([pm25_df, pm25_pom_df], axis=1)
    elif mdf['key'][0] == 'PM10':
        pm10_pom_df = chart_data
        pm10_pom_df.rename(columns={'value':sensor_df['sensorId'][idx].astype(str)}, inplace=True)
        pm10_df = pd.concat([pm10_df, pm10_pom_df], axis=1)
    else:
        continue
        

Processing data to line chart

In [10]:
def process_df(df):
    
    #renamimg the first column to 'data'
    df.rename(columns={df.columns[0]: 'data'}, inplace=True)
    #removing duplicate columns
    df = df.loc[:, ~df.columns.duplicated()]
    #computing the average excluding the 'data' column
    avg = df.loc[:, df.columns != 'data'].mean(axis=1)
    #creating a new DataFrame with 'data' and 'avg' columns
    result_df = pd.concat([df[['data']], avg.rename('avg')], axis=1)
    #resetting the index
    result_df.reset_index(drop=True, inplace=True)
    
    return result_df

In [11]:
measurement_df.reset_index(drop=True, inplace=True)

o3_df = process_df(o3_df)
no2_df = process_df(no2_df)
so2_df = process_df(so2_df)
pm25_df = process_df(pm25_df)
pm10_df = process_df(pm10_df)

full_df = pd.merge(ssaq_df, measurement_df, on='sensorId')
full_df

,stationId,stationName,gegrLat,gegrLon,addressStreet,city.id,city.name,city.commune.communeName,city.commune.districtName,city.commune.provinceName,...,pm10IndexLevel.id,pm10IndexLevel.indexLevelName,pm25IndexLevel.id,pm25IndexLevel.indexLevelName,o3IndexLevel,no2IndexLevel,stIndexLevel,key,date,value
0,114,"Wrocław, ul. Bartnicza",51.115933,17.141125,ul. Bartnicza,1064,Wrocław,Wrocław,Wrocław,DOLNOŚLĄSKIE,...,Brak danych,Brak danych,Brak danych,Brak danych,Brak danych,Brak danych,Brak danych,NO2,2024-05-23 15:00:00,3.23765
1,114,"Wrocław, ul. Bartnicza",51.115933,17.141125,ul. Bartnicza,1064,Wrocław,Wrocław,Wrocław,DOLNOŚLĄSKIE,...,Brak danych,Brak danych,Brak danych,Brak danych,Brak danych,Brak danych,Brak danych,O3,2024-05-23 15:00:00,102.69600
2,117,"Wrocław, wyb. Conrada-Korzeniowskiego",51.129378,17.029250,ul. Wyb. J.Conrada-Korzeniowskiego 18,1064,Wrocław,Wrocław,Wrocław,DOLNOŚLĄSKIE,...,1.0,Dobry,0.0,Bardzo dobry,Brak danych,Brak danych,Brak danych,SO2,2024-05-23 15:00:00,12.79310
3,117,"Wrocław, wyb. Conrada-Korzeniowskiego",51.129378,17.029250,ul. Wyb. J.Conrada-Korzeniowskiego 18,1064,Wrocław,Wrocław,Wrocław,DOLNOŚLĄSKIE,...,1.0,Dobry,0.0,Bardzo dobry,Brak danych,Brak danych,Brak danych,C6H6,2024-05-23 15:00:00,0.12960
4,117,"Wrocław, wyb. Conrada-Korzeniowskiego",51.129378,17.029250,ul. Wyb. J.Conrada-Korzeniowskiego 18,1064,Wrocław,Wrocław,Wrocław,DOLNOŚLĄSKIE,...,1.0,Dobry,0.0,Bardzo dobry,Brak danych,Brak danych,Brak danych,CO,2024-05-23 15:00:00,425.04300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,10934,"Kołobrzeg, ul. Żółkiewskiego",54.179324,15.596342,ul. Żółkiewskiego,386,Kołobrzeg,Kołobrzeg,kołobrzeski,ZACHODNIOPOMORSKIE,...,0.0,Bardzo dobry,0.0,Bardzo dobry,Brak danych,Brak danych,Brak danych,PM2.5,2024-05-23 15:00:00,4.18611
708,10934,"Kołobrzeg, ul. Żółkiewskiego",54.179324,15.596342,ul. Żółkiewskiego,386,Kołobrzeg,Kołobrzeg,kołobrzeski,ZACHODNIOPOMORSKIE,...,0.0,Bardzo dobry,0.0,Bardzo dobry,Brak danych,Brak danych,Brak danych,PM10,2024-05-23 15:00:00,4.55111
709,17158,"Dąbki, ul. Sztormowa",54.377492,16.310728,Sztormowa,155,Dąbki,Darłowo,sławieński,ZACHODNIOPOMORSKIE,...,0.0,Bardzo dobry,0.0,Bardzo dobry,Brak danych,Brak danych,Brak danych,NO2,2024-05-23 15:00:00,1.87695
710,17158,"Dąbki, ul. Sztormowa",54.377492,16.310728,Sztormowa,155,Dąbki,Darłowo,sławieński,ZACHODNIOPOMORSKIE,...,0.0,Bardzo dobry,0.0,Bardzo dobry,Brak danych,Brak danych,Brak danych,PM2.5,2024-05-23 15:00:00,12.12190


Processing data (casting, renaming) 

In [12]:
full_df.rename(columns={'gegrLat':'lat', 'gegrLon':'lon', 'key':'zwiazek'}, inplace=True)
full_df['city.commune.provinceName'] = full_df['city.commune.provinceName'].str.capitalize()
full_df['lat'] = full_df['lat'].astype(str).astype(float)
full_df['lon'] = full_df['lon'].astype(str).astype(float)
full_df['stationId'] = full_df['stationId'].astype(str).astype(int)
full_df['stCalcDate'] = pd.to_datetime(full_df['stCalcDate'])
full_df['stSourceDataDate'] = pd.to_datetime(full_df['stSourceDataDate'])
full_df['no2IndexLevel.id'] = full_df['no2IndexLevel.id'].astype(str)
full_df['stIndexLevel.id'] = full_df['stIndexLevel.id'].astype(str)

full_df.index.name = 'idx'
chart_data_df.index.name = 'idx'

full_df

,stationId,stationName,lat,lon,addressStreet,city.id,city.name,city.commune.communeName,city.commune.districtName,city.commune.provinceName,...,pm10IndexLevel.id,pm10IndexLevel.indexLevelName,pm25IndexLevel.id,pm25IndexLevel.indexLevelName,o3IndexLevel,no2IndexLevel,stIndexLevel,zwiazek,date,value
idx,,,,,,,,,,,,,,,,,,,,,
0,114,"Wrocław, ul. Bartnicza",51.115933,17.141125,ul. Bartnicza,1064,Wrocław,Wrocław,Wrocław,Dolnośląskie,...,Brak danych,Brak danych,Brak danych,Brak danych,Brak danych,Brak danych,Brak danych,NO2,2024-05-23 15:00:00,3.23765
1,114,"Wrocław, ul. Bartnicza",51.115933,17.141125,ul. Bartnicza,1064,Wrocław,Wrocław,Wrocław,Dolnośląskie,...,Brak danych,Brak danych,Brak danych,Brak danych,Brak danych,Brak danych,Brak danych,O3,2024-05-23 15:00:00,102.69600
2,117,"Wrocław, wyb. Conrada-Korzeniowskiego",51.129378,17.029250,ul. Wyb. J.Conrada-Korzeniowskiego 18,1064,Wrocław,Wrocław,Wrocław,Dolnośląskie,...,1.0,Dobry,0.0,Bardzo dobry,Brak danych,Brak danych,Brak danych,SO2,2024-05-23 15:00:00,12.79310
3,117,"Wrocław, wyb. Conrada-Korzeniowskiego",51.129378,17.029250,ul. Wyb. J.Conrada-Korzeniowskiego 18,1064,Wrocław,Wrocław,Wrocław,Dolnośląskie,...,1.0,Dobry,0.0,Bardzo dobry,Brak danych,Brak danych,Brak danych,C6H6,2024-05-23 15:00:00,0.12960
4,117,"Wrocław, wyb. Conrada-Korzeniowskiego",51.129378,17.029250,ul. Wyb. J.Conrada-Korzeniowskiego 18,1064,Wrocław,Wrocław,Wrocław,Dolnośląskie,...,1.0,Dobry,0.0,Bardzo dobry,Brak danych,Brak danych,Brak danych,CO,2024-05-23 15:00:00,425.04300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,10934,"Kołobrzeg, ul. Żółkiewskiego",54.179324,15.596342,ul. Żółkiewskiego,386,Kołobrzeg,Kołobrzeg,kołobrzeski,Zachodniopomorskie,...,0.0,Bardzo dobry,0.0,Bardzo dobry,Brak danych,Brak danych,Brak danych,PM2.5,2024-05-23 15:00:00,4.18611
708,10934,"Kołobrzeg, ul. Żółkiewskiego",54.179324,15.596342,ul. Żółkiewskiego,386,Kołobrzeg,Kołobrzeg,kołobrzeski,Zachodniopomorskie,...,0.0,Bardzo dobry,0.0,Bardzo dobry,Brak danych,Brak danych,Brak danych,PM10,2024-05-23 15:00:00,4.55111
709,17158,"Dąbki, ul. Sztormowa",54.377492,16.310728,Sztormowa,155,Dąbki,Darłowo,sławieński,Zachodniopomorskie,...,0.0,Bardzo dobry,0.0,Bardzo dobry,Brak danych,Brak danych,Brak danych,NO2,2024-05-23 15:00:00,1.87695


# Export data to .csv

In [13]:
full_df.to_excel("/arcgis/home/air_data_tabelka.xlsx")
pm10_df.to_excel("/arcgis/home/pm10_tabelka.xlsx")
pm25_df.to_excel("/arcgis/home/pm25_tabelka.xlsx")
so2_df.to_excel("/arcgis/home/so2_tabelka.xlsx") 
o3_df.to_excel("/arcgis/home/o3_tabelka.xlsx") 
no2_df.to_excel("/arcgis/home/no2_tabelka.xlsx")

full_df.to_csv("/arcgis/home/air_data.csv")
pm10_df.to_csv("/arcgis/home/pm10_df.csv")
pm25_df.to_csv("/arcgis/home/pm25_data.csv")
so2_df.to_csv("/arcgis/home/so2_data.csv") 
o3_df.to_csv("/arcgis/home/o3_data.csv") 
no2_df.to_csv("/arcgis/home/no2_data.csv") 

# Update data

In [14]:
flayercol.manager.overwrite("/arcgis/home/air_data.csv")
flc_pm10.manager.overwrite("/arcgis/home/pm10_df.csv")
flc_pm25.manager.overwrite("/arcgis/home/pm25_data.csv")
flc_so2.manager.overwrite("/arcgis/home/so2_data.csv")
flc_o3.manager.overwrite("/arcgis/home/o3_data.csv")
flc_no2.manager.overwrite("/arcgis/home/no2_data.csv")

{'success': True}

# Cities layer

In [15]:
cities = full_df[['lon','lat','city.name']]
cities = cities.groupby('city.name', as_index=False).first()
cities.reset_index(drop=True, inplace=True)
cities.index.name = 'idx'
cities.to_csv("/arcgis/home/cities.csv")
flc_cities.manager.overwrite("/arcgis/home/cities.csv")

{'success': True}